In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
bat_data = pd.read_csv("Clean_Bat.csv",index_col = "Unnamed: 0")
bat_data.fillna(0,inplace=True)
bat_data.drop_duplicates(inplace=True)
pitch_data = pd.read_csv("Clean_Pitch.csv",index_col = "Unnamed: 0")
pitch_data.fillna(0,inplace=True)
pitch_data.drop_duplicates(inplace=True)
prior_games = pd.read_csv("Previous_Games.csv",index_col="Unnamed: 0")
rolling_window_bat = pd.read_csv("7_Day_Window_Bat.csv",index_col="Unnamed: 0")
rolling_window_pitch = pd.read_csv("7_Day_Window_Pitch.csv",index_col="Unnamed: 0")

In [3]:
total_sum_df = pd.DataFrame()
start_date = prior_games.Date.unique()[0]
rolling_window_df = pd.DataFrame()
x = list(prior_games.Date.unique()).index(start_date)
while x <= len(prior_games.Date.unique()):
    team_date_df = pd.DataFrame()
    if x < 7:
        x+=7
    print(prior_games.Date.unique()[x-1])
    date_list = prior_games.Date.unique()[x-7:x]
    for date in date_list:
        date_df = bat_data[bat_data['Date'] == date]
        team_df = pd.DataFrame()
        for team in date_df.Team.unique():
            temp_df = pd.DataFrame(date_df.groupby("Team").get_group(team)[['PA','AB','R','H','2B','3B','HR','RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']].sum()).T
            temp_df['Home_Away'] = int(date_df.Home_Away.iloc[-1])
            temp_df['Date'] = date_df.Date.iloc[0]
            temp_df['Team'] = team
            temp_df.set_index("Team",inplace=True)
            team_df = pd.concat([team_df,temp_df])
        team_date_df = pd.concat([team_date_df,team_df])
    team_date_df.reset_index(inplace=True)
    for team in team_date_df.Team.unique():
        team_df = team_date_df
        team_df = team_df[team_df['Team'] == team]
        temp_df = pd.DataFrame(team_df.sum()[['PA','AB','R','H','2B','3B','HR','RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']]).T
        temp_df['Date'] = team_df.iloc[-1]['Date']
        temp_df['Team'] = team
        temp_df['Home_Away'] = team_df.iloc[-1].Home_Away
        total_sum_df = pd.concat([total_sum_df,temp_df])
    x+=1
if len(total_sum_df) > 0:
    final_df = pd.concat([rolling_window_df,total_sum_df])
    final_df.to_csv("7_Day_Window_Bat.csv")

2022-04-13
2022-04-14
2022-04-15
2022-04-16
2022-04-17
2022-04-18
2022-04-19
2022-04-20
2022-04-21
2022-04-22
2022-04-23
2022-04-24
2022-04-25
2022-04-26
2022-04-27
2022-04-28
2022-04-29
2022-04-30
2022-05-01
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-07
2022-05-08
2022-05-09
2022-05-10
2022-05-11
2022-05-12
2022-05-13
2022-05-14
2022-05-15
2022-05-16
2022-05-17
2022-05-18
2022-05-19
2022-05-20
2022-05-21
2022-05-22
2022-05-23
2022-05-24
2022-05-25
2022-05-26
2022-05-27
2022-05-28
2022-05-29
2022-05-30
2022-05-31
2022-06-01
2022-06-02
2022-06-03
2022-06-04
2022-06-05
2022-06-06
2022-06-07
2022-06-08
2022-06-09
2022-06-10
2022-06-11
2022-06-12
2022-06-13
2022-06-14
2022-06-15
2022-06-16
2022-06-17
2022-06-18
2022-06-19
2022-06-20
2022-06-21
2022-06-22
2022-06-23
2022-06-24
2022-06-25
2022-06-26
2022-06-27
2022-06-28
2022-06-29
2022-06-30
2022-07-01
2022-07-02
2022-07-03
2022-07-04
2022-07-05
2022-07-06
2022-07-07
2022-07-08
2022-07-09
2022-07-10
2022-07-11
2022-07-12

In [4]:
total_sum_df = pd.DataFrame()
start_date = prior_games.Date.unique()[0]
rolling_window_df = pd.DataFrame()
x = list(prior_games.Date.unique()).index(start_date)
while x <= len(prior_games.Date.unique()):
    team_date_df = pd.DataFrame()
    if x < 7:
        x+=7
    print(prior_games.Date.unique()[x-1])
    date_list = prior_games.Date.unique()[x-7:x]
    for date in date_list:
        date_df = pitch_data[pitch_data['Date'] == date]
        team_df = pd.DataFrame()
        for team in date_df.Team.unique():
            temp_df = pd.DataFrame(date_df.groupby("Team").get_group(team)[['GS', 'IP', 'H', 'R', 'ER','BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB','CS', 'PO', 'BF', 'GB/FB', 'LD', 'PU']].sum()).T
            temp_df['Home_Away'] = int(date_df.Home_Away.iloc[-1])
            temp_df['Date'] = date_df.Date.iloc[-1]
            temp_df['Team'] = team
            temp_df.set_index("Team",inplace=True)
            team_df = pd.concat([team_df,temp_df])
        team_date_df = pd.concat([team_date_df,team_df])
    team_date_df.reset_index(inplace=True)
    for team in team_date_df.Team.unique():
        team_df = team_date_df
        team_df = team_df[team_df['Team'] == team]
        temp_df = pd.DataFrame(team_df.sum()[['GS', 'IP', 'H', 'R', 'ER','BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB','CS', 'PO', 'BF', 'GB/FB', 'LD', 'PU']]).T
        temp_df['Date'] = team_df.iloc[-1]['Date']
        temp_df['Team'] = team
        temp_df['Home_Away'] = team_df.iloc[-1].Home_Away
        total_sum_df = pd.concat([total_sum_df,temp_df])
    x+=1
if len(total_sum_df) > 0:
    final_df = pd.concat([rolling_window_df,total_sum_df])
    final_df.drop(columns = ['GB/FB','LD','PU'],inplace=True)
    final_df.to_csv("7_Day_Window_Pitch.csv")

2022-04-13
2022-04-14
2022-04-15
2022-04-16
2022-04-17
2022-04-18
2022-04-19
2022-04-20
2022-04-21
2022-04-22
2022-04-23
2022-04-24
2022-04-25
2022-04-26
2022-04-27
2022-04-28
2022-04-29
2022-04-30
2022-05-01
2022-05-02
2022-05-03
2022-05-04
2022-05-05
2022-05-06
2022-05-07
2022-05-08
2022-05-09
2022-05-10
2022-05-11
2022-05-12
2022-05-13
2022-05-14
2022-05-15
2022-05-16
2022-05-17
2022-05-18
2022-05-19
2022-05-20
2022-05-21
2022-05-22
2022-05-23
2022-05-24
2022-05-25
2022-05-26
2022-05-27
2022-05-28
2022-05-29
2022-05-30
2022-05-31
2022-06-01
2022-06-02
2022-06-03
2022-06-04
2022-06-05
2022-06-06
2022-06-07
2022-06-08
2022-06-09
2022-06-10
2022-06-11
2022-06-12
2022-06-13
2022-06-14
2022-06-15
2022-06-16
2022-06-17
2022-06-18
2022-06-19
2022-06-20
2022-06-21
2022-06-22
2022-06-23
2022-06-24
2022-06-25
2022-06-26
2022-06-27
2022-06-28
2022-06-29
2022-06-30
2022-07-01
2022-07-02
2022-07-03
2022-07-04
2022-07-05
2022-07-06
2022-07-07
2022-07-08
2022-07-09
2022-07-10
2022-07-11
2022-07-12

In [ ]:
total_sum_df = pd.DataFrame()


In [4]:
total_df = pd.DataFrame()
for team in bat_data.sort_values("Team").Team.unique():
    team_df = bat_data[bat_data['Team'] == team]
    date_list = team_df.Date.unique()
    for date in team_df.Date.unique():
        date_df = team_df[team_df['Date'] == date]
        sum_day = pd.DataFrame(date_df[['PA','AB','R','H','2B','3B','HR','RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']].sum()).T
        sum_day['Team'] = date_df.iloc[0]['Team']
        sum_day['Home_Away'] = int(date_df.iloc[0]['Home_Away'])
        sum_day['Date'] = date_df.iloc[0]['Date']
        sum_day['G'] = 1
        total_df = pd.concat([total_df,sum_day])
total_df.head()

,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,...,HBP,SH,SF,GDP,SB,CS,Team,Home_Away,Date,G
0,34,26,4,3,0,0,1,3,7,0,...,1,0,0,1,0,0,ARI,0,2022-04-07,1
0,30,29,0,2,0,0,0,0,1,0,...,0,0,0,0,0,0,ARI,0,2022-04-08,1
0,33,31,2,5,2,0,1,2,2,0,...,0,0,0,1,0,0,ARI,0,2022-04-09,1
0,39,30,5,5,1,0,2,5,9,0,...,0,0,0,2,1,0,ARI,0,2022-04-10,1
0,36,30,1,4,1,0,1,1,5,0,...,0,1,0,0,0,0,ARI,0,2022-04-12,1


In [5]:
total_df[total_df['Team'] == "ARI"]

,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,...,HBP,SH,SF,GDP,SB,CS,Team,Home_Away,Date,G
0,34,26,4,3,0,0,1,3,7,0,...,1,0,0,1,0,0,ARI,0,2022-04-07,1
0,30,29,0,2,0,0,0,0,1,0,...,0,0,0,0,0,0,ARI,0,2022-04-08,1
0,33,31,2,5,2,0,1,2,2,0,...,0,0,0,1,0,0,ARI,0,2022-04-09,1
0,39,30,5,5,1,0,2,5,9,0,...,0,0,0,2,1,0,ARI,0,2022-04-10,1
0,36,30,1,4,1,0,1,1,5,0,...,0,1,0,0,0,0,ARI,0,2022-04-12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,42,33,5,11,3,0,0,5,7,1,...,0,1,1,0,1,0,ARI,0,2023-06-16,1
0,33,31,6,8,0,0,4,6,2,0,...,0,0,0,2,0,0,ARI,0,2023-06-17,1
0,42,35,3,9,3,0,1,3,7,0,...,0,0,0,1,0,0,ARI,0,2023-06-18,1
0,42,38,9,11,2,0,2,8,4,0,...,0,0,0,0,0,0,ARI,1,2023-06-19,1


In [3]:
import pandas as pd
import numpy as np
import warnings
from time import sleep
warnings.filterwarnings("ignore")
from datetime import datetime,timedelta
from pybaseball import batting_stats_range,pitching_stats_range,playerid_reverse_lookup

In [ ]:
temp_bat_df = batting_stats_range("2022-04-07","2022-04-08")
temp_bat_df.head()

In [ ]:
temp_bat_df.columns

In [ ]:
start_date = "2022-04-07"
start_date = datetime.strptime(start_date,"%Y-%m-%d")
next_week = start_date + timedelta(days=7)
temp_bat_df = batting_stats_range(start_date.strftime("%Y-%m-%d"),next_week.strftime("%Y-%m-%d"))
temp_bat_df['Date'] = next_week.strftime("%Y-%m-%d")
temp_bat_df.head()

In [7]:
start_date = "2022-04-14"
now = datetime.now()
end_date = now.strftime("%Y-%m-%d")
start_date = datetime.strptime(start_date,"%Y-%m-%d")
total_bat_df = pd.DataFrame()
total_pitch_df = pd.DataFrame()
now = now - timedelta(days = 1)
while start_date < now:
    print(start_date)
    sleep(10)
    next_day = start_date + timedelta(days=1)
    next_week = start_date - timedelta(days=7)
    temp_bat_df = batting_stats_range(next_week.strftime("%Y-%m-%d"),start_date.strftime("%Y-%m-%d"))
    temp_pitch_df = pitching_stats_range(next_week.strftime("%Y-%m-%d"),start_date.strftime("%Y-%m-%d"))
    if start_date.strftime("%Y-%m-%d") == "2022-10-05":
        start_date = "2023-04-06"
    else:
        start_date = next_day
    temp_bat_df['Date'] = next_week.strftime("%Y-%m-%d")
    temp_pitch_df['Date'] = next_week.strftime("%Y-%m-%d")
    data = playerid_reverse_lookup(temp_bat_df['mlbID'].unique(),key_type = "mlbam")
    last_list = []
    first_list = []
    for x in range(len(data)):
        last_list.append(data.iloc[x]['name_last'].capitalize())
        first_list.append(data.iloc[x]['name_first'].capitalize())
    data['name_last'] = last_list
    data['name_first'] = first_list
    data['Name'] = data['name_first'] + " " + data['name_last']
    key_dict = {data['key_mlbam'][i]: data['Name'][i] for i in range(len(data))}
    for name in temp_bat_df['mlbID']:
        try:
            key_dict[name]
        except:
            key_dict[name] = temp_bat_df[temp_bat_df['mlbID'] == name]['Name']
    temp_bat_df['mlbID'] = temp_bat_df['mlbID'].apply(lambda x: key_dict[x])
    temp_bat_df['Name'] = temp_bat_df['mlbID']

    names = temp_pitch_df['mlbID'].unique()
    int_names = [int(name) for name in names]
    data = playerid_reverse_lookup(int_names,key_type = "mlbam")
    last_list = []
    first_list = []
    for x in range(len(data)):
        last_list.append(data.iloc[x]['name_last'].capitalize())
        first_list.append(data.iloc[x]['name_first'].capitalize())
    data['name_last'] = last_list
    data['name_first'] = first_list
    data['Name'] = data['name_first'] + " " + data['name_last']
    data['key_mlbam'] = [str(name) for name in data['key_mlbam']]
    key_dict = {data['key_mlbam'][i]: data['Name'][i] for i in range(len(data))}
    for name in temp_pitch_df['mlbID']:
        try:
            key_dict[name]
        except:
            key_dict[name] = temp_pitch_df[temp_pitch_df['mlbID'] == name]['Name']
    temp_pitch_df['mlbID'] = temp_pitch_df['mlbID'].apply(lambda x: key_dict[x])
    temp_pitch_df['Name'] = temp_pitch_df['mlbID']
    temp_bat_df['Team'] = temp_bat_df.Lev + temp_bat_df.Tm
    temp_pitch_df['Team'] = temp_pitch_df.Lev + temp_pitch_df.Tm
    team_dict = {'Maj-NLMilwaukee':"MIL",
                 'Maj-ALLos Angeles': "LAA",
                 'Maj-NLAtlanta': "ATL",
                 'Maj-NLArizona': "ARI",
                 'Maj-NLNew York': "NYN",
                 'Maj-ALHouston': "HOU",
                 'Maj-NLCincinnati':"CIN",
                 'Maj-NLSt. Louis': "SLN",
                 'Maj-NLSan Diego': "SDN",
                 'Maj-NLWashington': "WSN",
                 'Maj-ALKansas City': "KCR",
                 'Maj-ALCleveland': "CLE",
                 'Maj-NLPittsburgh':"PIT",
                 'Maj-NLChicago':"CHN", 
                 'Maj-ALChicago':"CHA",
                 'Maj-NLMiami':"MIA",
                 'Maj-ALOakland':"OAK",
                 'Maj-ALBoston':"BOS",
                 'Maj-ALTampa Bay':"TBA",
                 'Maj-ALMinnesota':"MIN",
                 'Maj-ALDetroit':"DET",
                 'Maj-NLSan Francisco':"SFN",
                 'Maj-NLLos Angeles':"LAN",
                 'Maj-ALToronto':"TOR",
                 'Maj-NLColorado':"COL",
                 'Maj-ALTexas':"TEX",
                 'Maj-NLPhiladelphia':"PHI",
                 'Maj-ALBaltimore':"BAL",
                 'Maj-ALSeattle':"SEA",
                 'Maj-ALNew York':"NYA",
                 'Maj-NLAtlanta,Chicago':"ATL"}
    for team in temp_pitch_df['Team'].unique():
        try:
            key_dict[name]
        except:
            
            ind = temp_pitch_df[temp_pitch_df['Team'] == team].index[0]
            team = team.split(",")[0]
            temp_pitch_df.iloc[ind]['Team'] = team
    temp_bat_df['Team'] = temp_bat_df['Team'].apply(lambda x: team_dict[x])
    temp_pitch_df['Team'] = temp_pitch_df['Team'].apply(lambda x: team_dict[x])
    temp_bat_df.drop(columns = ["Lev","Tm","mlbID"],inplace = True)
    temp_pitch_df.drop(columns = ["Lev","Tm","mlbID"],inplace = True)
    total_bat_df = pd.concat([total_bat_df,temp_bat_df])
    total_pitch_df = pd.concat([total_pitch_df,temp_pitch_df])
total_bat_df['Home_Away'] = total_bat_df[total_bat_df.columns[4]]
total_pitch_df['Home_Away'] = total_pitch_df[total_pitch_df.columns[4]]
bat_important_stats = ['Name','Date','Opp','Team','Home_Away','PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS']
pitch_important_stats = ['Name','Date','Opp','Team','Home_Away','GS','IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB', 'CS', 'PO', 'BF', 'GB/FB', 'LD', 'PU']
total_bat_df = total_bat_df[bat_important_stats]
total_pitch_df = total_pitch_df[pitch_important_stats]
bat_df = pd.concat([bat_df,total_bat_df])
pitch_df = pd.concat([pitch_df,total_pitch_df])
bat_df['Home_Away'] = bat_df['Home_Away'].replace('@',1)
bat_df['Home_Away'] = bat_df['Home_Away'].replace('', 0) 
bat_df['Date'] = pd.to_datetime(bat_df['Date'])
pitch_df['Home_Away'] = pitch_df['Home_Away'].replace('@',1)
pitch_df['Home_Away'] = pitch_df['Home_Away'].replace('', 0) 
pitch_df['Date'] = pd.to_datetime(pitch_df['Date'])
pitch_df.to_csv("7_Days_Pitch.csv")
bat_df.to_csv("7_Days_Bat.csv")

2022-04-14 00:00:00


IndexError: list index out of range

In [10]:
temp_pitch_df[temp_pitch_df['Team'] == team]

'Maj-NLChicago'

In [14]:
temp_pitch_df[temp_pitch_df['Team'] == "Maj-NLAtlanta,Chicago"].index

Int64Index([80, 311], dtype='int64')

temp_pitch_df.Team.unique()